In [1]:
import os
import random
import pandas as pd
from moviepy.editor import *
import moviepy.audio.fx.all as afx
from moviepy.video import fx
from moviepy.config import change_settings
change_settings({"FFMPEG_BINARY":"ffmpeg"})
# from PIL import Image

In [2]:
select_folder = int(input('Select folder: '))

In [3]:
if select_folder == 1:
    folder_test = 'History'
elif select_folder == 2:
    folder_test = 'Music'
elif select_folder == 3:
    folder_test = 'Tech'
else:
    pass

In [4]:
carpeta = f'/mnt/D8E84E4DE84E2A58/Env_python/Create_video_news/4_create_video_by_news/videos_by_news'
intro_path = f'/mnt/D8E84E4DE84E2A58/backUp_Img_Vid/Intro/{folder_test}/Intro.mp4'
video_transicion_path = "transicion.png"
excel_path = 'titulo_entre_videos.xlsx'

In [5]:
def combinar_videos_con_fade(carpeta, excel_path, num_archivos=20, duracion_fade=1, duracion_transicion=4, transicion_path="transicion.png"):
    # Seleccionar archivos de video en orden
    archivos = sorted([f for f in os.listdir(carpeta) if f.endswith('.mp4')])
    if len(archivos) == 0:
        raise ValueError(f'La carpeta {carpeta} está vacía o no contiene archivos válidos.')

    # Leer archivo Excel
    if not os.path.exists(excel_path):
        raise FileNotFoundError(f"No se encontró el archivo Excel en {excel_path}")
    df = pd.read_excel(excel_path)

    # Filtrar el DataFrame para que coincida con los nombres completos de los archivos
    df = df[df['conca'].isin(archivos)]
    if df.empty:
        raise ValueError("No se encontraron coincidencias entre los videos y el archivo Excel.")

    # Limitar la cantidad de archivos
    archivos = archivos[:num_archivos]
    ruta_archivos = [os.path.join(carpeta, archivo) for archivo in archivos]

    # Verificar existencia de la transición
    if not os.path.exists(transicion_path):
        raise FileNotFoundError(f'No se encontró la imagen de transición en {transicion_path}')

    # Crear clips y añadir transiciones personalizadas
    clips_con_transiciones = []
    for i, ruta in enumerate(ruta_archivos):
        # Crear texto para la transición
        nombre_video = os.path.basename(ruta)
        titulo = df.loc[df['conca'] == nombre_video, 'Titulo'].values
        if titulo.size > 0 and not pd.isnull(titulo[0]):
            titulo_texto = str(titulo[0])
            texto_clip = TextClip(
                titulo_texto,
                fontsize=50,  # Tamaño base de fuente
                color='white',
                font="Arial",
                size=(1920, 1080),  # Tamaño del cuadro
                method='caption',  # Ajuste automático del texto
                align='center'  # Alineación del texto dentro del cuadro
            ).set_duration(duracion_transicion)

        else:
            texto_clip = None

        # Crear transición combinando la imagen y el texto
        transicion_imagen = ImageClip(transicion_path, duration=duracion_transicion)
        if texto_clip:
            transicion = CompositeVideoClip([transicion_imagen, texto_clip.set_position('center')])
        else:
            transicion = transicion_imagen

        transicion = transicion.fadein(duracion_fade).fadeout(duracion_fade)

        # Cargar video principal
        clip = VideoFileClip(ruta).fadein(duracion_fade).fadeout(duracion_fade)
        clips_con_transiciones.append(clip)

        # Añadir la transición combinada
        clips_con_transiciones.append(transicion)

    # Concatenar todos los clips (videos + transiciones)
    video_final = concatenate_videoclips(clips_con_transiciones, method="compose")
    return video_final


Este error específico indica que ImageMagick está bloqueando ciertas operaciones debido a su configuración de seguridad predeterminada. Este problema suele ocurrir con versiones recientes de ImageMagick, donde algunas acciones, como la creación de clips de texto, están deshabilitadas por el archivo policy.xml.
Solución para el problema
1. Editar el archivo policy.xml de ImageMagick

El archivo de configuración policy.xml se encuentra en el directorio de instalación de ImageMagick. Necesitamos modificarlo para permitir las operaciones que MoviePy requiere. Sigue estos pasos:
Ubicación del archivo policy.xml:

    Linux/Ubuntu:
    Usualmente en /etc/ImageMagick-6/ o /etc/ImageMagick-7/.
    MacOS:
    Usualmente en /usr/local/etc/ImageMagick-7/.
    Windows:
    Ubícalo en el directorio de instalación de ImageMagick (algo como C:\Program Files\ImageMagick-7.X.X-Q16-HDRI\policy.xml).

Modificación del archivo:

    Abre el archivo policy.xml con permisos de administrador:
        En Linux:

    sudo nano /etc/ImageMagick-7/policy.xml

    En Windows: Usa un editor de texto como Notepad++ con permisos de administrador.

Busca las líneas relacionadas con @ o TXT. Deberían lucir algo como esto:

<policy domain="path" rights="none" pattern="@*" />

Cambia el valor de rights="none" a rights="read|write", permitiendo a MoviePy acceder a esos recursos:

    <policy domain="path" rights="read|write" pattern="@*" />

    Guarda el archivo y cierra el editor.

2. Verifica la instalación de ImageMagick

Asegúrate de que ImageMagick esté correctamente instalado y accesible desde la línea de comandos:

    Ejecuta el siguiente comando en la terminal o línea de comandos:

    magick -version

    Si ImageMagick está instalado correctamente, deberías ver la versión instalada.

3. Reinicia tu sistema (opcional)

En algunos sistemas operativos, es posible que necesites reiniciar el sistema para que los cambios en el archivo policy.xml tengan efecto.
4. Configura el binario de ImageMagick en MoviePy

Para asegurarte de que MoviePy esté usando el binario correcto de ImageMagick, configura explícitamente la ruta en tu código:

from moviepy.config import change_settings

change_settings({"IMAGEMAGICK_BINARY": "ruta/a/magick"})  # Cambia "ruta/a/magick" a la ubicación exacta del binario

Por ejemplo:

    En Linux/MacOS: "magick"
    En Windows: "C:/Program Files/ImageMagick-7.X.X-Q16-HDRI/magick.exe"

5. Ejecuta nuevamente tu código

Después de realizar estos cambios, vuelve a ejecutar el script. Ahora MoviePy debería poder generar los clips de texto sin errores.

In [6]:
# Bloque de uso
try:
    # Combinar videos con transiciones
    video_final = combinar_videos_con_fade(carpeta, excel_path, num_archivos=10, duracion_fade=2)

    # Cargar intro y combinar con el video final
    intro = VideoFileClip(intro_path).set_duration(5)  # Ajustar duración del intro si es necesario
    final_video = concatenate_videoclips([intro, video_final], method="compose")

    # Guardar el video final
    final_video.write_videofile('video_automatico.mp4', codec="h264_nvenc", audio_codec="aac", fps=24)
    print("¡Listo! Video automático creado con éxito.")
except ValueError as e:
    print(f"Error: {e}")
except FileNotFoundError as e:
    print(f"No se encontró el archivo: {e}")
except Exception as e:
    print(f"Error inesperado: {e}")

Moviepy - Building video video_automatico.mp4.
MoviePy - Writing audio in video_automaticoTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video video_automatico.mp4



t:   1%|          | 86/10394 [00:01<03:13, 53.34it/s, now=None]

KeyboardInterrupt: 